In [1]:
import json, sys, os
import numpy as np
import pandas as pd
from enum import Enum
from scipy.optimize import root_scalar
from typing import Dict, Tuple, Union, Optional
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)
print("Added to sys.path:", repo_root)
from fixedincomelib import *
print("Fixed Income Library is loaded.")

Added to sys.path: /Users/lunli/Documents/FixedIncomeLib
Fixed Income Library is loaded.


In [2]:
# utilities
def display_res(input_dict : Dict[SimpleMetrics, float]):
    display({k.to_string() : v for k, v in input_dict.items()})

### Test all sorts sabr conversions

In [14]:
### hagan's classic formula
alpha = 0.1108019639023036
beta = 0.6
nu = 0.5
rho = 0.5
shift = 0.04
forward = 0.04
strike = 0.042
tte = 0.5

res_imp_log_normal_vol =  qfEuropeanOptionSABRLogNormalSigma( \
    forward, strike, tte, alpha, beta, rho, nu, shift, True)
display_res(res_imp_log_normal_vol)

{'implied_log_normal_vol': 0.30976746070686645,
 'd_ln_sigma_d_alpha': 2.7839279973131683,
 'd_ln_sigma_d_beta': -0.7704537904713582,
 'd_ln_sigma_d_forward': -2.399854599361494,
 'd_ln_sigma_d_strike': 0.8460014428403783,
 'd_ln_sigma_d_nu': 0.01789508576496479,
 'd_ln_sigma_d_rho': 0.0048162714097429995,
 'd_ln_sigma_d_tte': 0.007644771640522098}

In [16]:
### b&r test
key = SimpleMetrics.IMPLIED_LOG_NORMAL_VOL
base_v = res_imp_log_normal_vol[key]

epsilon = 1e-6
bump_vec = [0.] * 7
for i in range(len(bump_vec)):
    bump_vec[i] += epsilon    
    bumped_v = qfEuropeanOptionSABRLogNormalSigma(
        forward + bump_vec[0], 
        strike + bump_vec[1], 
        tte + bump_vec[2], 
        alpha + bump_vec[3], 
        beta + bump_vec[4], 
        rho + bump_vec[5], 
        nu + bump_vec[6], 
        shift, 
        False)
    display((bumped_v[key] - base_v) / epsilon)
    bump_vec[i] -= epsilon

-2.4046363872876064

0.8413163561926673

0.007644771649051307

2.783928167049421

-0.7728611928770057

0.004816266274687564

0.01789509251892696

In [17]:
### alpha to atm log-normal sigma
ln_sigma_atm = 0.1108019639023036
beta = 0.6
nu = 0.5
rho = 0.5
shift = 0.04
forward = 0.04
strike = 0.04
tte = 0.5

res_alpha_imp =  qfEuropeanOptionSABRAlphaFromATMLogNormalSigma( \
    forward, tte, ln_sigma_atm, beta, rho, nu, shift, True)
display_res(res_alpha_imp)

{'alpha': 0.04,
 'd_alpha_d_ln_sigma_atm': 0.36024000755240065,
 'd_alpha_d_beta': 0.10060611406874262,
 'd_alpha_d_nu': -0.001193618510973938,
 'd_alpha_d_rho': 0.0004553208990984172,
 'd_alpha_d_forward': 0.2006113676737952,
 'd_alpha_d_tte': -0.0006815089425002967}

In [18]:
### b&r test
key = SabrMetircs.ALPHA
base_v = res_alpha_imp[key]

epsilon = 1e-6
bump_vec = [0.] * 6
for i in range(len(bump_vec)):
    bump_vec[i] += epsilon    
    bumped_v = qfEuropeanOptionSABRAlphaFromATMLogNormalSigma(
        forward + bump_vec[0], 
        tte + bump_vec[1], 
        ln_sigma_atm + bump_vec[2], 
        beta + bump_vec[3], 
        rho + bump_vec[4], 
        nu + bump_vec[5], 
        shift, 
        False)
    display((bumped_v[key] - base_v) / epsilon)
    bump_vec[i] -= epsilon

0.19999925000152574

-0.0006815089256817508

0.36024000055895344

0.10090137354118456

0.00045532152354033073

-0.0011936195018491524

In [19]:
### alpha from atm normal sigma
sigma_nv = 0.008
beta = 0.6
nu = 0.5
rho = 0.5
forward = 0.04
tte = 0.5
shift = 0.04

res_nv_to_alpha = qfEuropeanOptionSABRAlphaFromATMNormalSigma( \
    forward, tte, sigma_nv, beta, rho, nu, shift, True)
display_res(res_nv_to_alpha)

{'alpha': 0.036115396338483506,
 'd_alpha_d_normal_sigma_atm': 4.507700224131979,
 'd_alpha_d_beta': 0.0908735826310518,
 'd_alpha_d_rho': 0.00042557541277180895,
 'd_alpha_d_nu': -0.0010638495041000173,
 'd_alpha_d_tte': -0.0005857192974776827,
 'd_alpha_d_forward': -0.26969444471709136}

In [20]:
### b&r test
key = SabrMetircs.ALPHA
base_v = res_nv_to_alpha[key]

epsilon = 1e-6
bump_vec = [0.] * 6
for i in range(len(bump_vec)):
    bump_vec[i] += epsilon    
    bumped_v = qfEuropeanOptionSABRAlphaFromATMNormalSigma(
        forward + bump_vec[0], 
        tte + bump_vec[1], 
        sigma_nv + bump_vec[2], 
        beta + bump_vec[3], 
        rho + bump_vec[4], 
        nu + bump_vec[5], 
        shift, 
        False)
    display((bumped_v[key] - base_v) / epsilon)
    bump_vec[i] -= epsilon

-0.27019034434111466

-0.0005857192092451058

4.50769948529689

0.09111286811136488

0.0004255759702975759

-0.0010638503966653445

In [21]:
### atm normal sigma from alpha
# from alpha to normal
alpha = res_nv_to_alpha[SabrMetircs.ALPHA]
beta = 0.6
nu = 0.5
rho = 0.5
forward = 0.04
tte = 0.5
shift = 0.04
res_alpha_to_nv = qfEuropeanOptionSABRAlphaFromATMNormalSigma( \
    forward, tte, alpha, beta, rho, nu, shift, True)
display(res_alpha_to_nv)

{<SabrMetircs.ALPHA: 'alpha'>: 0.16255319957848316,
 <SabrMetircs.D_ALPHA_D_NORMAL_SIGMA_ATM: 'd_alpha_d_normal_sigma_atm'>: 4.496739708984385,
 <SabrMetircs.D_ALPHA_D_BETA: 'd_alpha_d_beta'>: 0.403116174825962,
 <SabrMetircs.D_ALPHA_D_RHO: 'd_alpha_d_rho'>: -0.0001769999307616879,
 <SabrMetircs.D_ALPHA_D_NU: 'd_alpha_d_nu'>: -0.0067832755879983,
 <SabrMetircs.D_ALPHA_D_TTE: 'd_alpha_d_tte'>: -0.0035433005515043874,
 <SabrMetircs.D_ALPHA_D_FORWARD: 'd_alpha_d_forward'>: -1.2072989183139775}

In [23]:
### b&r test
key = SabrMetircs.ALPHA
base_v = res_alpha_to_nv[key]

epsilon = 1e-6
bump_vec = [0.] * 6
for i in range(len(bump_vec)):
    bump_vec[i] += epsilon    
    bumped_v = qfEuropeanOptionSABRAlphaFromATMNormalSigma(
        forward + bump_vec[0], 
        tte + bump_vec[1], 
        alpha + bump_vec[2], 
        beta + bump_vec[3], 
        rho + bump_vec[4], 
        nu + bump_vec[5], 
        shift, 
        False)
    display((bumped_v[key] - base_v) / epsilon)
    bump_vec[i] -= epsilon

-1.2172376726016232

-0.0035435635536984478

4.496732775832735

0.40888049146481364

-0.00017699747223431928

-0.006783279354882765